## Generative Deep Learning

In [1]:
# auto reloading
%reload_ext autoreload
%autoreload 2

In [2]:
# import
import numpy as np
import keras
from myutils import *

/home/atul/anaconda/envs/dpl/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
text = open('../../datasets/nietzsche.txt').read().lower()

In [4]:
print("Corpus length: %s" % str(len(text)))

Corpus length: 600893


In [5]:
max_len = 60  # seq of characters to train
step = 3      # 
sents = []
next_char = []

In [6]:
for i in range(0, len(text)-max_len, step):
    sents.append(text[i: i+max_len])
    next_char.append(text[i+max_len])

In [7]:
sents[:3]

['preface\n\n\nsupposing that truth is a woman--what then? is the',
 'face\n\n\nsupposing that truth is a woman--what then? is there ',
 'e\n\n\nsupposing that truth is a woman--what then? is there not']

In [8]:
next_char[:3]

['r', 'n', ' ']

In [9]:
text[:180]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand wo'

In [10]:
st = "this is me this calling atul"

In [11]:
word_index = word_index_generator(text)
word_index

{'privileges': 0,
 'why': 1,
 'class': 2,
 'talent:': 3,
 'world-exposition': 4,
 'intellects,': 5,
 'pretence,': 6,
 'usefulness:--all': 7,
 'attain': 8,
 'fear"': 9,
 'nostrils,': 10,
 'precisely': 11,
 'remarked,': 12,
 'phenomena,': 13,
 'teacher': 14,
 'forth.': 15,
 'subjecting': 16,
 'neck,': 17,
 'vigilance': 18,
 'scientificness': 19,
 '90': 20,
 '=reverence': 21,
 'good-will,': 22,
 'character,': 23,
 'severity;': 24,
 'unholiness': 25,
 'congratulates': 26,
 'confidingness,': 27,
 'younger': 28,
 'finer': 29,
 'spectacle,': 30,
 'limitation': 31,
 'decay.': 32,
 'playing,': 33,
 'newspaper-reading,': 34,
 'thereby,': 35,
 'sayest': 36,
 'communicative': 37,
 'determiner': 38,
 'auxiliaries;': 39,
 'remotest': 40,
 'accuracy': 41,
 'angels': 42,
 'sanctifying,': 43,
 'rights"': 44,
 'despection.': 45,
 'compunction': 46,
 'generative': 47,
 'one";': 48,
 'self-adapting,': 49,
 'do"--that': 50,
 'loathing,': 51,
 'restless': 52,
 'colours': 53,
 'function': 54,
 'overmasters':

In [25]:
char_index = char_index_generator(text)
char_index

{'\n': 29,
 ' ': 0,
 '!': 38,
 '"': 13,
 "'": 11,
 '(': 19,
 ')': 39,
 ',': 36,
 '-': 47,
 '.': 9,
 '0': 33,
 '1': 40,
 '2': 8,
 '3': 5,
 '4': 45,
 '5': 26,
 '6': 12,
 '7': 44,
 '8': 15,
 '9': 32,
 ':': 10,
 ';': 30,
 '=': 41,
 '?': 54,
 '[': 31,
 ']': 35,
 '_': 50,
 'a': 28,
 'b': 37,
 'c': 18,
 'd': 16,
 'e': 2,
 'f': 7,
 'g': 20,
 'h': 23,
 'i': 34,
 'j': 56,
 'k': 6,
 'l': 46,
 'm': 14,
 'n': 4,
 'o': 25,
 'p': 24,
 'q': 27,
 'r': 17,
 's': 55,
 't': 49,
 'u': 51,
 'v': 21,
 'w': 1,
 'x': 3,
 'y': 43,
 'z': 48,
 'ä': 52,
 'æ': 22,
 'é': 42,
 'ë': 53}

In [31]:
class mytest(object):
    def __init__(self, name, class_):
        self.name = name
        self.class_ = class_
        print("Hi %s"%self.name)

In [32]:
mine = mytest('atul', 12)

Hi atul


In [29]:
mine.name

'atul'

In [30]:
mine.class_

12

In [33]:
import json

In [35]:
strg = '{"name":"wala"}'

json.loads(strg)['name']

'wala'

In [36]:
from flask import request

In [37]:
abc = "[
('callback', 'jQuery111208315887275421632_1521463760575'), 
('[{\"doc_type\":\"text\",\"doc_name\":\"learnings.txt\"}]', ''), 
('_', '1521463760577')]"

In [40]:
mine.name[-2:]

'ul'